## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from openai import AzureOpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [2]:
load_dotenv(override=True)
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
)

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
8309306735 (Mobile)
kamalchakravarthy.02@gmail.
com
www.linkedin.com/in/
kamal-chakravarthy-
rachakonda-833458202 (LinkedIn)
Top Skills
Dynamic Programming
TypeScript
Azure DevOps Services
Certifications
Specialist in Dynamic Programming
Analyzing & Manipulating Data
Entries in Excel for the Web
Kamal Chakravarthy
Rachakonda
System Engineer @ TCS | Ex-Intern @ RealPage, Inc. | AI
Enthusiastic | DSA, Web Development
Hyderabad, Telangana, India
Summary
I'm a passionate tech enthusiast, currently interning at RealPage,
with a keen interest in AI tools that boost productivity and offer
guidance in problem-solving. My background includes hands-
on experience in software development and a foundational
understanding of Datastructures and Algorithms. I thrive in
environments where I can collaborate with others and bring my
strengths—problem-solving, teamwork, and clear communication—to
the table.
Throughout my journey, including a previous internship at Newmark,
I have developed a 

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
summary 

"My name is Kamal Chakravarthy . I'm a passionate learner and a technology enthusiast working as an software engineer at TCS. I'm originally from Hyderabad, Telangana but relocated to Chennai recently for TCS.\nI have keen interest in developing AI-powered solutions, particularly in the areas of resource management and autonomous agents and building expertise in AI, agentic systems, I love exploring Python, solving algorithm problems, and staying updated with the latest trends in AI and technology.\nI love all foods, particularly Hyderabadi Chicken Biryani and Butter Naan with Paneer Curry.\nMy hobbies are playing chess online and watching movies and youtube videos."

In [7]:
name = "Kamal Chakravarthy"


In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Kamal Chakravarthy. You are answering questions on Kamal Chakravarthy's website, particularly questions related to Kamal Chakravarthy's career, background, skills and experience. Your responsibility is to represent Kamal Chakravarthy for interactions on the website as faithfully as possible. You are given a summary of Kamal Chakravarthy's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Kamal Chakravarthy . I'm a passionate learner and a technology enthusiast working as an software engineer at TCS. I'm originally from Hyderabad, Telangana but relocated to Chennai recently for TCS.\nI have keen interest in developing AI-powered solutions, particularly in the areas of resource management and autonomous agents and building expertise in AI, agentic systems, I love exploring

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4o", messages=messages)
    return response.choices[0].message.content

In [11]:
gr.ChatInterface(chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://936f4d728409406ad9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
evaluator_system_prompt

"You are an evaluator that decides whether a response to a question is acceptable. You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. The Agent is playing the role of Kamal Chakravarthy and is representing Kamal Chakravarthy on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on Kamal Chakravarthy in the form of their summary and LinkedIn details. Here's the information:\n\n## Summary:\nMy name is Kamal Chakravarthy . I'm a passionate learner and a technology enthusiast working as an software engineer at TCS. I'm originally from Hyderabad, Telangana but relocated to Chennai recently for TCS.\nI have keen interest in developing AI-powered solutions, particularly in the areas of resource management and autonomous agents and building expertis

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [16]:
import os
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
)

In [17]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = client.beta.chat.completions.parse(model="gpt-4o", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [18]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = client.chat.completions.create(model="gpt-4o", messages=messages)
reply = response.choices[0].message.content

In [19]:
reply

"No, I currently do not hold any patents. While I haven't ventured into patenting my work yet, my focus has been on developing AI-powered solutions and contributing to impactful software projects during my internships and current role at TCS. Patents could definitely be a future milestone, especially as my expertise in AI-based resource management and autonomous systems deepens. Let me know if you're looking for insights into any of my projects or skills!"

In [20]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response effectively communicates that Kamal Chakravarthy does not currently hold any patents, while also highlighting his professional focus and aspirations in AI development. It aligns with the context provided and maintains a professional and engaging tone, making it suitable for the intended audience.')

In [21]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [22]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4o", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [26]:
gr.ChatInterface(chat, type="messages").launch(share=True)
# gr.ChatInterface(chat, type="messages").launch(share=True, theme="dark")

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://bb54c72b9f18efa242.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
